In [ ]:
# | default_exp _code_generator.app_generator

In [ ]:
# | export

from typing import *
import time
import json
from pathlib import Path

from yaspin import yaspin

from fastkafka_gen._components.logger import get_logger
from fastkafka_gen._code_generator.helper import CustomAIChat, ValidateAndFixResponse, write_file_contents, read_file_contents, validate_python_code
from fastkafka_gen._code_generator.prompts import APP_GENERATION_PROMPT
from fastkafka_gen._code_generator.constants import ASYNC_API_SPEC_FILE_NAME, APPLICATION_FILE_NAME

In [ ]:

from tempfile import TemporaryDirectory

from fastkafka_gen._components.logger import suppress_timestamps

In [ ]:
# | export

logger = get_logger(__name__)

In [ ]:
suppress_timestamps()
logger = get_logger(__name__, level=20)
logger.info("ok")

[INFO] __main__: ok


In [ ]:
# | export


def generate_app(code_gen_directory: str) -> Dict[str, int]:
    """Generate code for the new FastKafka app from the validated plan

    Args:
        code_gen_directory: The directory containing the generated files.

    Returns:
        The total token used to generate the FastKafka code
    """
    # TODO: Validate the generated code
    with yaspin(
        text="Generating FastKafka app...", color="cyan", spinner="clock"
    ) as sp:
        spec_file_name = f"{code_gen_directory}/{ASYNC_API_SPEC_FILE_NAME}"
        asyncapi_spec = read_file_contents(spec_file_name)

        app_generator = CustomAIChat(
            params={
                "temperature": 0.5,
            },
            user_prompt=APP_GENERATION_PROMPT,
        )
        app_validator = ValidateAndFixResponse(app_generator, validate_python_code)
        validated_app, usage = app_validator.fix(asyncapi_spec)

        output_file = f"{code_gen_directory}/{APPLICATION_FILE_NAME}"
        write_file_contents(output_file, validated_app)

        sp.text = ""
        sp.ok(f" ✔ FastKafka app generated and saved at: {output_file}")
        return usage

In [ ]:
# | notest

fixture_spec = '''
asyncapi: 2.5.0
info:
  title: Product currency converter
  version: 0.0.1
  description: 'A FastKafka application using localhost broker for testing, staging.airt.ai
    for staging and prod.airt.ai for production, using default port numbers. It should
    consume from ''store_product'' topic an JSON encoded object with the following
    three attributes: product_name, currency and price. The format of the currency
    will be three letter string, e.g. ''EUR''. For each consumed message, check if
    the currency attribute is set to ''HRK''. If it is then change the currency to
    ''EUR'' and divide the price by 7.5, if the currency is not set to ''HRK'' don''t
    change the original message. Finally, publish the consumed message to ''change_currency''
    topic. Use SASL_SSL with SCRAM-SHA-256 for authentication.'
  contact:
    name: Author
    url: https://www.google.com/
    email: noreply@gmail.com
servers:
  localhost:
    url: localhost
    description: local development kafka broker
    protocol: kafka
    variables:
      port:
        default: '9092'
  staging:
    url: staging.airt.ai
    description: staging kafka broker
    protocol: kafka-secure
    security:
    - staging_default_security: []
    variables:
      port:
        default: '9092'
  production:
    url: prod.airt.ai
    description: production kafka broker
    protocol: kafka-secure
    security:
    - production_default_security: []
    variables:
      port:
        default: '9092'
channels:
  store_product:
    subscribe:
      message:
        $ref: '#/components/messages/StoreProduct'
      description: For each consumed message, check if the currency attribute is set
        to 'HRK'. If it is then change the currency to 'EUR', then make a new instance of the message and only change the price attribute
        by dividing it by 7.5, if the currency is not set to 'HRK' don't change the original message.
        Finally, publish the consumed message to 'change_currency' topic.
  change_currency:
    publish:
      message:
        $ref: '#/components/messages/StoreProduct'
components:
  messages:
    StoreProduct:
      payload:
        properties:
          product_name:
            description: Name of the product.
            title: Product Name
            type: string
          currency:
            description: The currency.
            title: Currency
            type: string
          price:
            description: Price of the product.
            title: Price
            type: number
        required:
        - product_name
        - currency
        - price
        title: StoreProduct
        type: object
  schemas: {}
  securitySchemes:
    staging_default_security:
      type: scramSha256
    production_default_security:
      type: scramSha256

'''

with TemporaryDirectory() as d:
    output_path = f"{str(d)}/fastkafka-gen"
    output_file = f"{output_path}/{APPLICATION_FILE_NAME}"
    spec_file = f"{output_path}/{ASYNC_API_SPEC_FILE_NAME}"    
    write_file_contents(spec_file, fixture_spec)
    
    usage = generate_app(output_path)
    
    assert Path(output_path).exists()
    
    actual = [file for file in Path(output_path).iterdir()]
    print(actual)
    assert len(actual) == 2
    
    contents = read_file_contents(output_file)
    print(contents)

assert int(usage["total_tokens"]) > 0
print(usage)

⠹ Generating FastKafka app... 

/home/robert/.local/lib/python3.11/site-packages/yaspin/core.py:59: UserWarning: color, on_color and attrs are not supported when running in jupyter
  self._color = self._set_color(color) if color else color


 ✔ FastKafka app generated and saved at: /tmp/tmpgptv6_yt/fastkafka-gen/application.py 
[PosixPath('/tmp/tmpgptv6_yt/fastkafka-gen/application.py'), PosixPath('/tmp/tmpgptv6_yt/fastkafka-gen/asyncapi.yml')]
from typing import *
from pydantic import BaseModel, Field
from aiokafka.helpers import create_ssl_context

from fastkafka import FastKafka


class StoreProduct(BaseModel):
    product_name: str = Field(..., description="Name of the product.")
    currency: str = Field(..., description="The currency.")
    price: float = Field(..., description="Price of the product.")

kafka_brokers = {
    "localhost": {
        "url": "localhost",
        "description": "local development kafka broker",
        "port": 9092,
    },
    "staging": {
        "url": "staging.airt.ai",
        "description": "staging kafka broker",
        "port": 9092,
        "protocol": "kafka-secure",
        "security": {"type": "scramSha256"},
    },
    "production": {
        "url": "prod.airt.ai",
        "de